In [1]:
from openpyxl import Workbook
import os

In [2]:
data = {}
# {'2018-12': {'card1': [Пришло, Ушло, Баланс в конце месяца, Дата и время], 'card2': [То же самое]}, ...}
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [3]:
# Из шаблона сообщения 480 вынимает date, card, change_b, balance
def read_480(array):
    global data
    date = array[2] + ' ' + array[3]
    card = array[6].strip(':') + ' ' + array[1]
    if array[4].strip(':') == 'Withdrawal':
        change_b = int(array[7]) * (-1)
    elif array[4].strip(':') == 'Transfer':
        change_b = int(array[7])
    balance = int(array[-2])
    return date, card, change_b, balance

# Из шаблона сообщения 720 вынимает date, card, change_b, balance
def read_720(array):
    global data
    date = array[2] + ' ' + array[3]
    card = array[4].strip(':') + ' ' + array[1]
    change_b = int(array[5])
    balance = int(array[-2])
    return date, card, change_b, balance

# Заполняет словарь data данными, полученными из датасета
def write_data(date_new, date, card, change_b, balance):
    global data
    if not date_new in data.keys():
        data[date_new] = {card: [change_b if change_b>0 else 0, abs(change_b) if change_b<=0 else 0, balance, date]}
    elif not card in data[date_new].keys():
        data[date_new][card] = [change_b if change_b>0 else 0, abs(change_b) if change_b<=0 else 0, balance, date]
    else:
        if date > data[date_new][card][3]:
            data[date_new][card][2] = balance # В data[date][card] лежит массив в котором 3 значение - последние дата и время в этом месяце, 
        if change_b > 0:                      # а 4 значение - баланс карты в это время
            data[date_new][card][0] += change_b
        else:
            data[date_new][card][1] -= change_b

# Единая функция вытаскивания данных из файла и заполнение по собственному шаблону data
def write_file(name_file):
    global data
    with open(name_file) as messages:
        for line in messages:
            try:
                line_array = line.split()

                if line_array[0] == '480':
                    date, card, change_b, balance = read_480(line_array)
                    # Из сообщения вытащили date, card, change_b, balance

                elif line_array[0] == '720':
                    date, card, change_b, balance = read_720(line_array)
                    # Из сообщения вытащили date, card, change_b, balance

                date_y_m = date[:7]
                write_data(date_y_m, date, card, change_b, balance)
                # Записали в data
            except:
                pass

# Закидывает данные в созданный эксель-документ и открывает его
def write_report(received, spent, month_id, year, card):
    global months
    wb = Workbook()
    ws = wb.active
    if month_id == -1: # Запрос на вывод данных по всем картам
        text = 'Report for ' + str(year)
        card = '*all (all_banks)'
    else:
        try:
            text = 'Report for ' + months[month_id] + ' ' + str(year) + ', card ' + card
        except:
            print(type(months[month_id]), type(year), type(card))
    array_data = [[text], ['Received', 'Spent', 'Delta'], [received, spent, received-spent]]
    for line in array_data:
        ws.append(line)
    name = 'report_for_' + str(year) + '_card_' + card.split()[0][1:] + '.xlsx'
    # Например name = 'report_for_2019_card_1234.xlsx'
    wb.save(name)
    os.startfile(name)
    print('Report created, opening ...')

In [4]:
name_file = 'message.txt'
# Шаблон этого файла: Номер телефона Название банка Дата Время Сообщение
# Всё разделено пробелами

write_file(name_file)
# Вытащили из датасета сообщения и записали чистую информацию в data

In [5]:
while True:
    print('Choose an option')
    print('1 - Show current funds')
    print('2 - Expenses per month')
    print('3 - Exit from the program')
    answer = input('Your choice: ')
    
    if answer == '3':
        break
        
    elif answer == '1':
        cards = []
        print('Your current funds:')
        total = 0
        for date in sorted(data, reverse = True):
            for card, mas in data[date].items():
                if not card in cards: # Выводит статистику по всем существующим картам
                    balance = mas[2]
                    total += balance
                    print(f'{card}: {balance} EUR')
                    cards.append(card)
        print(f'Total: {total} EUR')
        pacifier = input('Press any key')
        
    elif answer == '2':
        end = False # Если меняет на True - выходит в главное меню
        while not end:
            date = input('Enter month and year in the following format YYYY-MM:')
            try:
                year = date.split('-')[0]
                month_id = int(date.split('-')[1]) - 1
                if date in data.keys():
                    cards = list(data[date].keys())
                    while True:
                        print('Select a credit card:')
                        k = 1
                        for card in cards:
                            print(f'{k} - {card}')
                            k += 1
                        print(f'{k} - Total')
                        print(f'{k+1} - Exit to the main menu')
                        answer = input('Your choice: ')
                        if answer == str(k+1):
                            end = True # Выходит из программы в главное меню
                            break
                        elif answer == str(k): # Считает статистику по всем картам в это
                            received_all = 0
                            spent_all = 0
                            for card in cards:
                                received_all += data[date][card][0]
                                spent_all += data[date][card][1]
                            print('Report for all cards')
                            print(f'Received: {received_all} EUR')
                            print(f'Spent: {spent_all} EUR')
                            if received_all > spent_all:
                                print(f'Delta: +{received_all-spent_all} EUR')
                            else:
                                print(f'Delta: {received_all-spent_all} EUR')
                            answer_2 = input('Export a full report to Excel? Write yes')
                            if answer_2.lower() == 'yes':
                                write_report(received_all, spent_all, -1, year, 0)
                        else:
                            try:
                                answer = int(answer)
                                print(f'Report for {months[month_id]} {year}, card {cards[answer-1]}')
                                received = data[date][cards[answer-1]][0]
                                spent = data[date][cards[answer-1]][1]
                                print(f'Received: {received} EUR')
                                print(f'Spent: {spent} EUR')
                                if received > spent:
                                    print(f'Delta: +{received-spent} EUR')
                                else:
                                    print(f'Delta: {received-spent} EUR')
                                answer_2 = input('Export a full report to Excel? Write yes')
                                if answer_2.lower() == 'yes':
                                    card_one = cards[answer-1]
                                    write_report(received, spent, month_id, year, card_one)
                            except:
                                print('Input error')
                else:
                    print('No spending this month')
                    print('Enter "1" to exit the main menu')
                    print('Or any other button for month and year introduction')
                    answer = input()
                    if answer == '1':
                        break
            except:
                print('Input error')
            
        
    
    else:
        print('Input error')

Choose an option
1 - Show current funds
2 - Expenses per month
3 - Exit from the program


Your choice:  1


Your current funds:
*1238 (GorgeousBank): 650 EUR
*6678 (SuperBank): 193 EUR
*6778 (SuperBank): 293 EUR
*1218 (GorgeousBank): 50 EUR
*6878 (SuperBank): 958 EUR
*1228 (GorgeousBank): 780 EUR
*6671 (SuperBank): 810 EUR
Total: 3734 EUR


Press any key 


Choose an option
1 - Show current funds
2 - Expenses per month
3 - Exit from the program


Your choice:  3
